In [1]:
%load_ext autoreload
%autoreload 2

# Importy

In [2]:
#from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
#from sklearn import tree
#from IPython.display import Image
#import pydotplus
#import graphviz
from utils import *

In [3]:
dataframes = read_csv("./datasets", 'csv')

In [4]:
res = prepare_dataframes(dataframes)

  0%|          | 0/3 [00:00<?, ?it/s]

out_hum
pressure
out_temp


In [5]:
df = get_labels(res, '2024-02-13')

In [6]:
df['rain_1h'] = df['rain_1h'].fillna(0)
df['snow_1h'] = df['snow_1h'].fillna(0)
df['total percipitation'] = df['rain_1h'] + df['snow_1h']

In [7]:
df['last_changed_count'] = df.groupby('last_changed')['last_changed'].transform('count')
df = df.drop_duplicates(subset=['last_changed'])
df = df.loc[:, ['last_changed', 'pressure', 'out_temp', 'out_hum', 'total percipitation']]
df = df.reset_index(drop=True)

## Tabela 1 (time, P, T, H, R)

In [8]:
df

,last_changed,pressure,out_temp,out_hum,total percipitation
0,2023-09-20 00:00:00+00:00,986.000000,15.300000,76.000000,0.00
1,2023-09-20 01:00:00+00:00,985.428571,14.528571,78.900000,0.00
2,2023-09-20 02:00:00+00:00,985.760000,14.212500,79.500000,0.00
3,2023-09-20 03:00:00+00:00,985.280000,14.041667,81.300000,0.00
4,2023-09-20 04:00:00+00:00,985.666667,13.833333,83.333333,0.25
...,...,...,...,...,...
3499,2024-02-12 19:00:00+00:00,974.777778,7.220000,85.166667,0.00
3500,2024-02-12 20:00:00+00:00,975.846154,7.020000,84.933333,0.00
3501,2024-02-12 21:00:00+00:00,976.846154,6.566667,84.033333,0.00
3502,2024-02-12 22:00:00+00:00,976.521739,6.370000,83.873913,0.00


In [9]:
df.value_counts('total percipitation')

total percipitation
0.00     2666
0.25      166
0.51      149
0.38       37
1.02       29
         ... 
1.53        1
1.59        1
1.61        1
1.66        1
30.99       1
Name: count, Length: 119, dtype: int64

## Tabela 2 (time, 0, -1, -2, -3...)

In [10]:
df2 = generate_data(df, 5)
df2

,time,TP t+1,Press t0,Temp t0,Hum t0,TP t0,Press t-1,Temp t-1,Hum t-1,TP t-1,...,Hum t-3,TP t-3,Press t-4,Temp t-4,Hum t-4,TP t-4,Press t-5,Temp t-5,Hum t-5,TP t-5
0,2023-09-20 05:00:00+00:00,0.0,986.217391,13.966667,83.366667,0.0,985.666667,13.833333,83.333333,0.25,...,79.500000,0.00,985.428571,14.528571,78.900000,0.00,986.000000,15.300000,76.000000,0.00
1,2023-09-20 06:00:00+00:00,0.0,986.478261,14.640000,80.100000,0.0,986.217391,13.966667,83.366667,0.00,...,81.300000,0.00,985.760000,14.212500,79.500000,0.00,985.428571,14.528571,78.900000,0.00
2,2023-09-20 07:00:00+00:00,0.0,986.739130,15.480000,75.420000,0.0,986.478261,14.640000,80.100000,0.00,...,83.333333,0.25,985.280000,14.041667,81.300000,0.00,985.760000,14.212500,79.500000,0.00
3,2023-09-20 08:00:00+00:00,0.0,986.090909,17.000000,69.840000,0.0,986.739130,15.480000,75.420000,0.00,...,83.366667,0.00,985.666667,13.833333,83.333333,0.25,985.280000,14.041667,81.300000,0.00
4,2023-09-20 09:00:00+00:00,0.0,986.200000,18.600000,61.920000,0.0,986.090909,17.000000,69.840000,0.00,...,80.100000,0.00,986.217391,13.966667,83.366667,0.00,985.666667,13.833333,83.333333,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,2024-02-12 18:00:00+00:00,0.0,974.642857,7.371429,84.830000,0.0,973.500000,7.540000,84.800000,0.00,...,83.640000,0.00,971.652174,7.860000,85.928571,0.00,971.130435,7.955556,86.000000,0.00
3494,2024-02-12 19:00:00+00:00,0.0,974.777778,7.220000,85.166667,0.0,974.642857,7.371429,84.830000,0.00,...,83.020000,0.00,972.384615,7.877778,83.640000,0.00,971.652174,7.860000,85.928571,0.00
3495,2024-02-12 20:00:00+00:00,0.0,975.846154,7.020000,84.933333,0.0,974.777778,7.220000,85.166667,0.00,...,84.800000,0.00,973.222222,7.754545,83.020000,0.00,972.384615,7.877778,83.640000,0.00
3496,2024-02-12 21:00:00+00:00,0.0,976.846154,6.566667,84.033333,0.0,975.846154,7.020000,84.933333,0.00,...,84.830000,0.00,973.500000,7.540000,84.800000,0.00,973.222222,7.754545,83.020000,0.00


In [11]:
X = df2.iloc[:,2:]
y = df2.iloc[:,1:2]

In [12]:
X

,Press t0,Temp t0,Hum t0,TP t0,Press t-1,Temp t-1,Hum t-1,TP t-1,Press t-2,Temp t-2,...,Hum t-3,TP t-3,Press t-4,Temp t-4,Hum t-4,TP t-4,Press t-5,Temp t-5,Hum t-5,TP t-5
0,986.217391,13.966667,83.366667,0.0,985.666667,13.833333,83.333333,0.25,985.280000,14.041667,...,79.500000,0.00,985.428571,14.528571,78.900000,0.00,986.000000,15.300000,76.000000,0.00
1,986.478261,14.640000,80.100000,0.0,986.217391,13.966667,83.366667,0.00,985.666667,13.833333,...,81.300000,0.00,985.760000,14.212500,79.500000,0.00,985.428571,14.528571,78.900000,0.00
2,986.739130,15.480000,75.420000,0.0,986.478261,14.640000,80.100000,0.00,986.217391,13.966667,...,83.333333,0.25,985.280000,14.041667,81.300000,0.00,985.760000,14.212500,79.500000,0.00
3,986.090909,17.000000,69.840000,0.0,986.739130,15.480000,75.420000,0.00,986.478261,14.640000,...,83.366667,0.00,985.666667,13.833333,83.333333,0.25,985.280000,14.041667,81.300000,0.00
4,986.200000,18.600000,61.920000,0.0,986.090909,17.000000,69.840000,0.00,986.739130,15.480000,...,80.100000,0.00,986.217391,13.966667,83.366667,0.00,985.666667,13.833333,83.333333,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,974.642857,7.371429,84.830000,0.0,973.500000,7.540000,84.800000,0.00,973.222222,7.754545,...,83.640000,0.00,971.652174,7.860000,85.928571,0.00,971.130435,7.955556,86.000000,0.00
3494,974.777778,7.220000,85.166667,0.0,974.642857,7.371429,84.830000,0.00,973.500000,7.540000,...,83.020000,0.00,972.384615,7.877778,83.640000,0.00,971.652174,7.860000,85.928571,0.00
3495,975.846154,7.020000,84.933333,0.0,974.777778,7.220000,85.166667,0.00,974.642857,7.371429,...,84.800000,0.00,973.222222,7.754545,83.020000,0.00,972.384615,7.877778,83.640000,0.00
3496,976.846154,6.566667,84.033333,0.0,975.846154,7.020000,84.933333,0.00,974.777778,7.220000,...,84.830000,0.00,973.500000,7.540000,84.800000,0.00,973.222222,7.754545,83.020000,0.00


In [13]:
y

,TP t+1
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
3493,0.0
3494,0.0
3495,0.0
3496,0.0


## Test 1: RFT

In [21]:
kfold = KFold(n_splits = 5)

classifiers = [    
    RandomForestRegressor(n_estimators=500, min_samples_leaf=100, min_samples_split=540),
    RandomForestRegressor(n_estimators=1000,min_samples_leaf=100, min_samples_split=540),
   
]

results_list = []
for classifier in tqdm(classifiers):
  
    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
        'n_estimators': classifier.get_params().get('n_estimators'),
        

    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_rft_1 = pd.DataFrame(results_list)

  0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
df_rft_1

,Classifier,min_samples_leaf,min_samples_split,n_estimators,r2,neg_mean_squared_error,neg_mean_absolute_error
0,RandomForestRegressor,100,540,500,0.114012,-0.903959,-0.230496
1,RandomForestRegressor,100,540,1000,0.115614,-0.904306,-0.229258


## Test 2: RFT

In [24]:
kfold = KFold(n_splits = 5)

classifiers = [    
    RandomForestRegressor(n_estimators=800, min_samples_leaf=100, min_samples_split=540),
    RandomForestRegressor(n_estimators=1500,min_samples_leaf=100, min_samples_split=540),
   
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
        'n_estimators': classifier.get_params().get('n_estimators'),
        
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_rft_2 = pd.DataFrame(results_list)

  0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
df_rft_2

,Classifier,min_samples_leaf,min_samples_split,n_estimators,r2,neg_mean_squared_error,neg_mean_absolute_error
0,RandomForestRegressor,100,540,800,0.116742,-0.904535,-0.229097
1,RandomForestRegressor,100,540,1500,0.116138,-0.904282,-0.229380


## Test 3: RFT

In [26]:
kfold = KFold(n_splits = 5)

classifiers = [    
    RandomForestRegressor(n_estimators=400, min_samples_leaf=100, min_samples_split=540),
    RandomForestRegressor(n_estimators=700,min_samples_leaf=100, min_samples_split=540),
   
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
        'n_estimators': classifier.get_params().get('n_estimators'),
        
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_rft_3 = pd.DataFrame(results_list)

  0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
df_rft_3 
#400	0.115807	-0.904697
#500	0.114012	-0.903959 !!!
#550	0.116518	-0.905241	
#600 	0.115338	-0.903864 !!!
#650    0.116429	-0.904310	
#700	0.114889	-0.904862
#800	0.116742	-0.904535
#1000	0.115614	-0.904306
#1500	0.116138	-0.904282

,Classifier,min_samples_leaf,min_samples_split,n_estimators,r2,neg_mean_squared_error,neg_mean_absolute_error
0,RandomForestRegressor,100,540,400,0.115807,-0.904697,-0.230706
1,RandomForestRegressor,100,540,700,0.114889,-0.904862,-0.229643


## Test 4: RFT

In [29]:
kfold = KFold(n_splits = 5)

classifiers = [    
    RandomForestRegressor(n_estimators=600, min_samples_leaf=100, min_samples_split=540),
   
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
        'n_estimators': classifier.get_params().get('n_estimators'),
        
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_rft_4 = pd.DataFrame(results_list)

  0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
df_rft_4

,Classifier,min_samples_leaf,min_samples_split,n_estimators,r2,neg_mean_squared_error,neg_mean_absolute_error
0,RandomForestRegressor,100,540,600,0.115338,-0.903864,-0.230248


## Test 5: RFT

In [31]:
kfold = KFold(n_splits = 5)

classifiers = [    
    RandomForestRegressor(n_estimators=550, min_samples_leaf=100, min_samples_split=540),
    RandomForestRegressor(n_estimators=650, min_samples_leaf=100, min_samples_split=540),

   
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
        'n_estimators': classifier.get_params().get('n_estimators'),
        
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_rft_5 = pd.DataFrame(results_list)

  0%|          | 0/2 [00:00<?, ?it/s]

In [32]:
df_rft_5

,Classifier,min_samples_leaf,min_samples_split,n_estimators,r2,neg_mean_squared_error,neg_mean_absolute_error
0,RandomForestRegressor,100,540,550,0.116518,-0.905241,-0.229389
1,RandomForestRegressor,100,540,650,0.116429,-0.904310,-0.230248


## Test 6: RFT

In [33]:
kfold = KFold(n_splits = 5)

classifiers = [    
    RandomForestRegressor(n_estimators=600, min_samples_leaf=50, min_samples_split=440),
    RandomForestRegressor(n_estimators=600, min_samples_leaf=150, min_samples_split=640),
   
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
        'n_estimators': classifier.get_params().get('n_estimators'),
        
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_rft_6 = pd.DataFrame(results_list)

  0%|          | 0/2 [00:00<?, ?it/s]

In [34]:
df_rft_6

,Classifier,min_samples_leaf,min_samples_split,n_estimators,r2,neg_mean_squared_error,neg_mean_absolute_error
0,RandomForestRegressor,50,440,600,0.108040,-0.90381,-0.228197
1,RandomForestRegressor,150,640,600,0.109962,-0.90888,-0.229527


## Test 7: RFT

In [35]:
kfold = KFold(n_splits = 5)

classifiers = [    
    RandomForestRegressor(n_estimators=600, min_samples_leaf=50, min_samples_split=540),
    RandomForestRegressor(n_estimators=600, min_samples_leaf=150, min_samples_split=540),
   
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
        'n_estimators': classifier.get_params().get('n_estimators'),
        
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_rft_7 = pd.DataFrame(results_list)

  0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
df_rft_7

,Classifier,min_samples_leaf,min_samples_split,n_estimators,r2,neg_mean_squared_error,neg_mean_absolute_error
0,RandomForestRegressor,50,540,600,0.105066,-0.903506,-0.227023
1,RandomForestRegressor,150,540,600,0.109484,-0.909204,-0.228891


## Test 8: RFT

In [37]:
kfold = KFold(n_splits = 5)

classifiers = [    
    RandomForestRegressor(n_estimators=600, min_samples_leaf=100, min_samples_split=540),
   
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
        'n_estimators': classifier.get_params().get('n_estimators'),
        

    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_rft_8 = pd.DataFrame(results_list)

  0%|          | 0/1 [00:00<?, ?it/s]

In [38]:
df_rft_8

,Classifier,min_samples_leaf,min_samples_split,n_estimators,r2,neg_mean_squared_error,neg_mean_absolute_error
0,RandomForestRegressor,100,540,600,0.114271,-0.904636,-0.229986


## Test 9: RFT

In [39]:
kfold = KFold(n_splits = 5)

classifiers = [    
    RandomForestRegressor(n_estimators=600, min_samples_leaf=100, min_samples_split=440),
    RandomForestRegressor(n_estimators=600, min_samples_leaf=100, min_samples_split=640), 
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
        'n_estimators': classifier.get_params().get('n_estimators'),
        

    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_rft_9 = pd.DataFrame(results_list)

  0%|          | 0/2 [00:00<?, ?it/s]

In [41]:
df_rft_9

,Classifier,min_samples_leaf,min_samples_split,n_estimators,r2,neg_mean_squared_error,neg_mean_absolute_error
0,RandomForestRegressor,100,440,600,0.113584,-0.904263,-0.229742
1,RandomForestRegressor,100,640,600,0.115414,-0.904556,-0.229481


## Test 10: RFT

In [42]:
kfold = KFold(n_splits = 5)

classifiers = [    
    RandomForestRegressor(n_estimators=600, min_samples_leaf=60, min_samples_split=540),
    RandomForestRegressor(n_estimators=600, min_samples_leaf=70, min_samples_split=540),
    RandomForestRegressor(n_estimators=600, min_samples_leaf=80, min_samples_split=540),
    RandomForestRegressor(n_estimators=600, min_samples_leaf=90, min_samples_split=540),

]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
        'n_estimators': classifier.get_params().get('n_estimators'),
        
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_rft_10 = pd.DataFrame(results_list)

  0%|          | 0/4 [00:00<?, ?it/s]

In [46]:
#50	540	600	0.105066	-0.903506	
#60	540	600	0.109919	-0.904053
#70	540	600	0.111150	-0.903877
#80	540	600	0.114994	-0.903157 !!!
#90	540	600	0.113263	-0.903392 
#100 540 600 0.114271	-0.904636	

In [43]:
df_rft_10

,Classifier,min_samples_leaf,min_samples_split,n_estimators,r2,neg_mean_squared_error,neg_mean_absolute_error
0,RandomForestRegressor,60,540,600,0.109919,-0.904053,-0.228871
1,RandomForestRegressor,70,540,600,0.111150,-0.903877,-0.228876
2,RandomForestRegressor,80,540,600,0.114994,-0.903157,-0.230000
3,RandomForestRegressor,90,540,600,0.113263,-0.903392,-0.229199


## Test 11: RFT

In [45]:
kfold = KFold(n_splits = 5)

classifiers = [    
    RandomForestRegressor(n_estimators=600, min_samples_leaf=90, min_samples_split=440),
    RandomForestRegressor(n_estimators=600, min_samples_leaf=90, min_samples_split=540),
    RandomForestRegressor(n_estimators=600, min_samples_leaf=90, min_samples_split=640),
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
        'n_estimators': classifier.get_params().get('n_estimators'),
        
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_rft_11 = pd.DataFrame(results_list)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#80
#

In [51]:
df_rft_11
#90	640	600	0.113575	-0.902970

,Classifier,min_samples_leaf,min_samples_split,n_estimators,r2,neg_mean_squared_error,neg_mean_absolute_error
0,RandomForestRegressor,90,440,600,0.115221,-0.903313,-0.229893
1,RandomForestRegressor,90,540,600,0.114956,-0.903381,-0.230308
2,RandomForestRegressor,90,640,600,0.113575,-0.902970,-0.229556


## Test 12: RFT

In [52]:
kfold = KFold(n_splits = 5)

classifiers = [    
    RandomForestRegressor(n_estimators=600, min_samples_leaf=80, min_samples_split=440),
    RandomForestRegressor(n_estimators=600, min_samples_leaf=80, min_samples_split=540),
    RandomForestRegressor(n_estimators=600, min_samples_leaf=80, min_samples_split=640),
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
        'n_estimators': classifier.get_params().get('n_estimators'),
        
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_rft_12 = pd.DataFrame(results_list)

  0%|          | 0/3 [00:00<?, ?it/s]

In [54]:
df_rft_12

,Classifier,min_samples_leaf,min_samples_split,n_estimators,r2,neg_mean_squared_error,neg_mean_absolute_error
0,RandomForestRegressor,80,440,600,0.113999,-0.903223,-0.229951
1,RandomForestRegressor,80,540,600,0.110978,-0.902985,-0.229844
2,RandomForestRegressor,80,640,600,0.112684,-0.903463,-0.229498


## Test 13: RFT

In [55]:
kfold = KFold(n_splits = 5)

classifiers = [    
    RandomForestRegressor(n_estimators=600, min_samples_leaf=80, min_samples_split=500),
    RandomForestRegressor(n_estimators=600, min_samples_leaf=80, min_samples_split=540),
    RandomForestRegressor(n_estimators=600, min_samples_leaf=80, min_samples_split=600),
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
        'n_estimators': classifier.get_params().get('n_estimators'),
        
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_rft_13 = pd.DataFrame(results_list)

  0%|          | 0/3 [00:00<?, ?it/s]

In [56]:
df_rft_13 
#80	440	600	0.113999	-0.903223	
#80	500	600	0.112318	-0.902948	 !!!
#80	540	600	0.110978	-0.902985	

,Classifier,min_samples_leaf,min_samples_split,n_estimators,r2,neg_mean_squared_error,neg_mean_absolute_error
0,RandomForestRegressor,80,500,600,0.112318,-0.902948,-0.229626
1,RandomForestRegressor,80,540,600,0.113446,-0.903481,-0.231035
2,RandomForestRegressor,80,600,600,0.115051,-0.902657,-0.229761


## Test 14: RFT

In [57]:
kfold = KFold(n_splits = 5)

classifiers = [    
    RandomForestRegressor(n_estimators=600, min_samples_leaf=80, min_samples_split=510),
    RandomForestRegressor(n_estimators=600, min_samples_leaf=80, min_samples_split=520),
    RandomForestRegressor(n_estimators=600, min_samples_leaf=80, min_samples_split=530),
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
        'n_estimators': classifier.get_params().get('n_estimators'),

    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_rft_14 = pd.DataFrame(results_list)

  0%|          | 0/3 [00:00<?, ?it/s]

In [58]:
df_rft_14
#80	440	600	0.113999	-0.903223	
#80	510	600	0.112033	-0.902276 !!!
#80	520	600	0.112634	-0.902620 
#80	530	600	0.112507	-0.903286	
#80	500	600	0.112318	-0.902948	 
#80	540	600	0.110978	-0.902985	

,Classifier,min_samples_leaf,min_samples_split,n_estimators,r2,neg_mean_squared_error,neg_mean_absolute_error
0,RandomForestRegressor,80,510,600,0.112033,-0.902276,-0.229368
1,RandomForestRegressor,80,520,600,0.112634,-0.902620,-0.230089
2,RandomForestRegressor,80,530,600,0.112507,-0.903286,-0.229353
